In [ ]:
import sys
sys.path.append("..")

import csaf
import csaf.utils as csafutils

from csaf_f16.goals import AcasSimpleScenario, AcasSimpleShieldScenario
from csaf_f16.acas import AcasScenarioViewer, collision_condition
from csaf_f16.systems import F16AcasShieldIntruderBalloon

from IPython.display import HTML

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# check the system
AcasSimpleScenario.system_type().check()

# view the system
csafutils.view_block_diagram(AcasSimpleScenario.system_type, ipython_notebook=True)

In [ ]:
YOFF = 1700
int_way = (-7000, 0.0+YOFF, 1000.0)

# construct a scenario
scen = AcasSimpleScenario()

In [ ]:
m = (8000+YOFF - int_way[1]) / (6000 - int_way[0])
theta = np.arctan2((6000 - int_way[0]), (8000+YOFF - int_way[1]))
x = 9000

sys = scen.generate_system([x, (x-int_way[0])*m+int_way[1], # relative position
                          (theta+np.pi), # relative heading
                          -200.0]) # relative airspeed
sys.check()

In [ ]:
trajs = sys.simulate_tspan((0.0, 60.0), show_status=True, terminating_conditions_all=collision_condition)

In [ ]:
viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
plt.show()

In [ ]:
anim = viewer.summary_video()
HTML(anim.to_jshtml())

In [ ]:
def task():
    c = np.random.random(4)
    c[0] *= 20000
    c[0] -= 10000
    c[1] *= 20000
    c[1] -= 10000
    c[2] *= np.pi * 2
    c[2] -= np.pi
    c[3] *= 0
    c[3] += 0.0
    sys = scen.generate_system(c) 
    trajs, passed = sys.simulate_tspan((0.0, 20.0), 
                               show_status=False, 
                               terminating_conditions_all=collision_condition,
                              return_passed = True)
    if not passed:
        return c, passed, trajs
    else:
        return c, passed, None

In [ ]:
task()

In [ ]:
from joblib import Parallel, delayed

In [ ]:
results = Parallel(n_jobs=24)(delayed(task)() for i in range(100))

In [ ]:
for c, p, t in results:
    if not p:
        viewer = AcasScenarioViewer(t, scen)
        fig, ax = viewer.summary_plot()
        plt.show()

In [ ]:
coords = np.array([c for c, _, _ in results])
passes = np.array([p for _, p, _ in results])


fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
x, y, z = coords[passes==True, 0], coords[passes==True, 1], coords[passes==True, 2]
ax.scatter(x, y, z)
x, y, z = coords[passes==False, 0], coords[passes==False, 1], coords[passes==False, 2]
ax.scatter(x, y, z, alpha=0.8)